## Loading packages

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

## Downloading data

In [2]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "dailyrainfall/"

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [4]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 4.51 s, sys: 6.63 s, total: 11.1 s
Wall time: 2min 19s


### Extracting ZIP

In [5]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 14.3 s, sys: 2.5 s, total: 16.8 s
Wall time: 17.1 s


In [6]:
%ls -ltr dailyrainfall/

total 12146232
-rw-r--r--   1 apple  staff  814041183 Mar 28 23:48 data.zip
-rw-r--r--   1 apple  staff   95376895 Mar 28 23:48 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff   94960113 Mar 28 23:48 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff   82474546 Mar 28 23:48 NorESM2-LM_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  127613760 Mar 28 23:48 ACCESS-CM2_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  232118894 Mar 28 23:48 FGOALS-f3-L_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  330360682 Mar 28 23:48 CMCC-CM2-HR4_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  254009247 Mar 28 23:48 MRI-ESM2-0_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  235661418 Mar 28 23:48 GFDL-CM4_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  294260911 Mar 28 23:48 BCC-CSM2-MR_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  295768615 Mar 28 23:48 EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 apple  staff  328852379 Mar 28 2

In [7]:
%%time
### just listing to get an idea how individual file looks like 
use_cols = ['time', 'lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']
df = pd.read_csv("dailyrainfall/NorESM2-LM_daily_rainfall_NSW.csv", usecols=use_cols)
df

CPU times: user 650 ms, sys: 71.7 ms, total: 722 ms
Wall time: 724 ms


,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-36.000000,-34.105263,141.25,143.75,7.651974e-07
1,1889-01-02 12:00:00,-36.000000,-34.105263,141.25,143.75,8.955062e-06
2,1889-01-03 12:00:00,-36.000000,-34.105263,141.25,143.75,1.229921e-03
3,1889-01-04 12:00:00,-36.000000,-34.105263,141.25,143.75,1.460834e-03
4,1889-01-05 12:00:00,-36.000000,-34.105263,141.25,143.75,1.145301e-09
...,...,...,...,...,...,...
919795,2014-12-27 12:00:00,-30.315789,-28.421053,151.25,153.75,5.602033e+00
919796,2014-12-28 12:00:00,-30.315789,-28.421053,151.25,153.75,2.049624e+01
919797,2014-12-29 12:00:00,-30.315789,-28.421053,151.25,153.75,7.168385e-02
919798,2014-12-30 12:00:00,-30.315789,-28.421053,151.25,153.75,1.460354e+00


In [10]:
%%time
## here we are using a normal python way for merging the data 
import pandas as pd
use_cols = ['time', 'lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']
files = glob.glob('dailyrainfall/*.csv')
files.remove('dailyrainfall/observed_daily_rainfall_SYD.csv')
df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
                .assign(model=re.findall(r'[^\/]+(?=\.)', file)[0].split("_")[0])
                for file in files)
              )
df.to_csv("dailyrainfall/combined_data.csv")

CPU times: user 5min 37s, sys: 14.6 s, total: 5min 52s
Wall time: 5min 55s


In [11]:
!du -sh dailyrainfall/combined_data.csv

5.6G	dailyrainfall/combined_data.csv


In [12]:
print(df.shape)
df.head()

(62467843, 6)


,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


### Time comparison between machines for combining data

Team Member | Operating System | RAM | Processor | Is SSD | Time taken
-- | -- | -- | -- | -- | --
Member 1 | MacOS Monterey  | 16Gb  | i5  | Yes | 5m 55s
Member 2 |   |   |   |   |  
Member 3 |   |   |   |   |  
Member 4 |   |   |   |   |  